# 5章
- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

In [1]:
# 5-1
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.9/336.9 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 55.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 46.4 MB/s eta 0:00:00
  Created wheel for fugashi: filename=fugashi-1.1.0-cp310-cp310-linux_x86_64.whl size=257492 sha256=1363de42b518b8b67632e104a5f2ba42d73e9f6709096b8787f5d905f62d5262
  Stored in directory: /root/.cache/pip/wheels/f6/bc/9b/6d38a64c5bea6582a87574f5c63b6c0bd9a4f5a6706ed577a5
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=d285af772ed278fc17739eefa84abf8fb04b38344d779f12919a39071f38c147
  Stored in di

In [2]:
# 5-2
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [3]:
# 5-3
model_name = 'tohoku-nlp/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Downloading:   0%|          | 0.00/252k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/120 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# 5-4
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


In [5]:
# 5-5
# 文章を符号化し、GPUに配置する。
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

# BERTに入力し、分類スコアを得る。
# 系列長を揃える必要がないので、単にiput_idsのみを入力します。
with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
    scores = output.logits

In [7]:
input_ids

tensor([[   2, 3246,    9,    4,  118, 3488,    8,    3]], device='cuda:0')

In [9]:
# 5-6
# ID列で'[MASK]'(IDは4)の位置を調べる
mask_position = input_ids[0].tolist().index(4)
print(mask_position)

# スコアが最も良いトークンのIDを取り出し、トークンに変換する。
id_best = scores[0, mask_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
print(token_best)
token_best = token_best.replace('##', '')
print(token_best)

# [MASK]を上で求めたトークンで置き換える。
text = text.replace('[MASK]',token_best)

print(text)

3
東京
東京
今日は東京へ行く。


In [12]:
# 5-7
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    """
    文章中の最初の[MASK]をスコアの上位のトークンに置き換える。
    上位何位まで使うかは、num_topkで指定。
    出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト。
    """
    # 文章を符号化し、BERTで分類スコアを得る。
    input_ids = tokenizer.encode(text, return_tensors='pt')
    print(input_ids)
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits

    # スコアが上位のトークンとスコアを求める。
    mask_position = input_ids[0].tolist().index(4)
    print(mask_position)
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices # トークンのID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # トークン
    scores_topk = topk.values.cpu().numpy() # スコア

    # 文章中の[MASK]を上で求めたトークンで置き換える。
    text_topk = [] # 穴埋めされたテキストを追加する。
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))

    return text_topk, scores_topk

text = '今日は[MASK]へ行く。'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

tensor([[   2, 3246,    9,    4,  118, 3488,    8,    3]])
3
今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


In [13]:
# 5-8
def greedy_prediction(text, tokenizer, bert_mlm):
    """
    [MASK]を含む文章を入力として、貪欲法で穴埋めを行った文章を出力する。
    """
    # 前から順に[MASK]を一つづつ、スコアの最も高いトークンに置き換える。
    for _ in range(text.count('[MASK]')):
        text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
    return text

text = '今日は[MASK][MASK]へ行く。'
greedy_prediction(text, tokenizer, bert_mlm)

tensor([[   2, 3246,    9,    4,    4,  118, 3488,    8,    3]])
3
tensor([[   2, 3246,    9,    6,    4,  118, 3488,    8,    3]])
4


'今日は、東京へ行く。'

In [14]:
# 5-9
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
greedy_prediction(text, tokenizer, bert_mlm)

tensor([[   2, 3246,    9,    4,    4,    4,    4,    4,    3]])
3
tensor([[   2, 3246,    9,  745,    4,    4,    4,    4,    3]])
4
tensor([[   2, 3246,    9,  745,  745,    4,    4,    4,    3]])
5
tensor([[   2, 3246,    9,  745,  745,   81,    4,    4,    3]])
6
tensor([[   2, 3246,    9,  745,  745,   81,   18,    4,    3]])
7


'今日は社会社会的な地位'

In [15]:
# 5-10
def beam_search(text, tokenizer, bert_mlm, num_topk):
    """
    ビームサーチで文章の穴埋めを行う。
    """
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        # 現在得られている、それぞれの文章に対して、
        # 最初の[MASK]をスコアが上位のトークンで穴埋めする。
        text_candidates = [] # それぞれの文章を穴埋めした結果を追加する。
        score_candidates = [] # 穴埋めに使ったトークンのスコアを追加する。
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append( score + scores_topk_inner )

        # 穴埋めにより生成された文章の中から合計スコアの高いものを選ぶ。
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [ text_candidates[idx] for idx in idx_list ]
        scores_topk = score_candidates[idx_list]

    return text_topk

text = "今日は[MASK][MASK]へ行く。"
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

tensor([[   2, 3246,    9,    4,    4,  118, 3488,    8,    3]])
3
tensor([[   2, 3246,    9,    6,    4,  118, 3488,    8,    3]])
4
tensor([[   2, 3246,    9, 1438,    4,  118, 3488,    8,    3]])
4
tensor([[   2, 3246,    9,   59,    4,  118, 3488,    8,    3]])
4
tensor([[   2, 3246,    9, 7755,    4,  118, 3488,    8,    3]])
4
tensor([[   2, 3246,    9,  552,    4,  118, 3488,    8,    3]])
4
tensor([[   2, 3246,    9,   73,    4,  118, 3488,    8,    3]])
4
tensor([[   2, 3246,    9,   70,    4,  118, 3488,    8,    3]])
4
tensor([[   2, 3246,    9, 1842,    4,  118, 3488,    8,    3]])
4
tensor([[   2, 3246,    9,  733,    4,  118, 3488,    8,    3]])
4
tensor([[   2, 3246,    9,  391,    4,  118, 3488,    8,    3]])
4
今日はお台場へ行く。
今日はお祭りへ行く。
今日はゲームセンターへ行く。
今日はお風呂へ行く。
今日はゲームショップへ行く。
今日は東京ディズニーランドへ行く。
今日はお店へ行く。
今日は同じ場所へ行く。
今日はあの場所へ行く。
今日は同じ学校へ行く。


In [16]:
# 5-11
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

tensor([[   2, 3246,    9,    4,    4,    4,    4,    4,    3]])
3
tensor([[   2, 3246,    9,  745,    4,    4,    4,    4,    3]])
4
tensor([[   2, 3246,    9, 1184,    4,    4,    4,    4,    3]])
4
tensor([[   2, 3246,    9,  886,    4,    4,    4,    4,    3]])
4
tensor([[   2, 3246,    9,  315,    4,    4,    4,    4,    3]])
4
tensor([[   2, 3246,    9, 2262,    4,    4,    4,    4,    3]])
4
tensor([[   2, 3246,    9,   53,    4,    4,    4,    4,    3]])
4
tensor([[    2,  3246,     9, 15249,     4,     4,     4,     4,     3]])
4
tensor([[   2, 3246,    9, 2609,    4,    4,    4,    4,    3]])
4
tensor([[   2, 3246,    9, 1617,    4,    4,    4,    4,    3]])
4
tensor([[   2, 3246,    9, 2282,    4,    4,    4,    4,    3]])
4
tensor([[   2, 3246,    9,  745,  745,    4,    4,    4,    3]])
5
tensor([[    2,  3246,     9, 15249,   745,     4,     4,     4,     3]])
5
tensor([[   2, 3246,    9,  315,  745,    4,    4,    4,    3]])
5
tensor([[   2, 3246,    9,   53,  745,    4,